#  Predicción de imágenes nuevas — Modelo CNN PlantVillage
Este notebook permite cargar el modelo entrenado (`modelo_cultivos.h5`) y realizar predicciones sobre imágenes nuevas de hojas.

##  Funciones principales
- Cargar el modelo y clases (`classes.npy`).
- Seleccionar una imagen desde tu computadora.
- Mostrar la imagen.
- Preprocesarla y obtener predicción con porcentaje de confianza.


In [ ]:
# 1️ Importar librerías
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tkinter import Tk, Button, Label, filedialog
from PIL import Image, ImageTk
import matplotlib.pyplot as plt

print('TensorFlow version:', tf.__version__)

In [ ]:
# 2️ Cargar modelo y clases
model = load_model('modelo_cultivos.h5')
classes = np.load('classes.npy', allow_pickle=True).item()
inv_classes = {v: k for k, v in classes.items()}
img_size = (128, 128)
print(f'Modelo cargado correctamente con {len(classes)} clases.')

In [ ]:
# 3️ Crear interfaz con Tkinter
root = Tk()
root.title('Predicción de Enfermedades en Hojas')
root.geometry('400x500')

label_img = Label(root)
label_img.pack(pady=10)

label_result = Label(root, text='', font=('Arial', 14))
label_result.pack(pady=10)

def seleccionar_imagen():
    file_path = filedialog.askopenfilename(
        filetypes=[('Imagenes', '*.jpg *.jpeg *.png *.bmp')]
    )
    if not file_path:
        return

    # Mostrar imagen seleccionada
    img = Image.open(file_path)
    img_resized = img.resize((250, 250))
    img_tk = ImageTk.PhotoImage(img_resized)
    label_img.config(image=img_tk)
    label_img.image = img_tk

    # Preprocesamiento
    img_array = image.load_img(file_path, target_size=img_size)
    img_array = image.img_to_array(img_array)
    img_array = np.expand_dims(img_array, axis=0) / 255.0

    # Predicción
    pred = model.predict(img_array)
    idx = np.argmax(pred)
    class_name = inv_classes[idx]
    confidence = pred[0][idx] * 100

    label_result.config(text=f'{class_name}\nConfianza: {confidence:.2f}%')

btn_select = Button(root, text='Seleccionar imagen', command=seleccionar_imagen, font=('Arial', 12))
btn_select.pack(pady=20)

root.mainloop()

 **Uso:**
1. Ejecuta todas las celdas.
2. Se abrirá una ventana.
3. Pulsa **Seleccionar imagen** y elige una hoja.
4. Verás la imagen y el resultado (nombre de la clase + porcentaje de confianza).
